In [ ]:
# Importing libraries
import pandas as pd


In [ ]:
import numpy as np


In [ ]:
import seaborn as sns


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
import seaborn as sns


In [ ]:
from sklearn.metrics import classification_report


In [ ]:
from sklearn import metrics


In [ ]:
from sklearn import tree


In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('Crop_recommendation.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df['label'].unique()

In [ ]:
df.dtypes

In [ ]:
df['label'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.corr(),annot=True)

### Seperating features and target label

In [ ]:
labels = ['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee']
labels_map = {labels[i]:i+1 for i in range(len(labels))}


In [ ]:
labels_map

In [ ]:
dummm = df.copy()


In [ ]:
df['label'] = df['label'].map(labels_map)


In [ ]:
df.head()

In [ ]:
sns.set_palette('Set3')
plt.figure(figsize=(16,9))
sns.countplot(data=df,x = 'label')

In [ ]:
import scipy.stats as stats

In [ ]:
def visualise(df,col):
    plt.figure(figsize=(15,7))
    plt.subplot(1,3,1)
    plt.hist(df[col],color = 'pink')
    
    plt.subplot(1,3,2)
    stats.probplot(df[col],dist='norm',plot=plt)
    
    plt.subplot(1,3,3)
    sns.boxplot(df[col],color = 'pink')
    
    plt.show()

In [ ]:
for i in df.columns[:-1]:
    print(f' Statistical plots for : {i} is shown ↓')
    visualise(df,i)
    print("-"*60)

In [ ]:
def quantile(df,val):
    a = []
    df_25 = df[val].quantile(0.25)
    df_75 = df[val].quantile(0.75)
    iqr = df_75-df_25
    upper_limit = float(df_75+(iqr*1.5))
    lower_limit = float(df_25-(iqr*1.5))
    for i in df[val]:
        if i>upper_limit:
            i = np.NaN
        elif i<lower_limit:
            i = np.NaN
        else:
            i = i
        a.append(i)
    return a
        
    

## Outlier Removal

### First Lets detect the outliers as shown ↓

**We will be using the gaussian trainsformation technique which says if there are datapoints greater than "mean+{num} * std",then there is a possibilty that that particular datapoint will not fit in the gaussian distribution,so here i have choosen the num as 3,so my range is {lower,upper} = {mean-{3} * std,mean+{3} * std},if there are any datapoints beyond these ranges,then they must be removed,so we find there are couple of data points of that,inorder to remove these outilers,we will be using another method of `Quantiles` which is `Inter Quantile Range` which is a better approach!**

In [ ]:
temp = df.copy()

In [ ]:
upper_list = []
lower_list = []

In [ ]:
for i in temp.columns[:-1]:
    upper_list.append(df[i].mean()+(df[i].std())*3)
    lower_list.append(df[i].mean()-(df[i].std())*3)

In [ ]:
j = 0

In [ ]:
for i in temp.columns[:-1]:
    dummy = temp[(temp[i]>upper_list[j])|(temp[i]<lower_list[j])];j+=1

In [ ]:
dummy

## Outlier Removal using IQR technique

In [ ]:
def quantile(df,val):
    a = []
    df_upper_limit = df[val].quantile(0.75)
    df_lower_limit = df[val].quantile(0.25)
    iqr = df_upper_limit-df_lower_limit
    upper_limit = float(df_upper_limit+(iqr)*1.5)
    lower_limit = float(df_lower_limit-(iqr)*1.5)
    
    for i in df[val]:
        if i>upper_limit:
            i = np.NaN
            
        elif i<lower_limit:
            i = np.NaN
            
        else:
            i = i
            
        a.append(i)
        
    return a
            
    

In [ ]:
df.columns

In [ ]:
temp = df.copy()


In [ ]:
temp['N'] = quantile(temp,'N')


In [ ]:
temp['P'] = quantile(temp,'P')


In [ ]:
temp['K'] = quantile(temp,'K')


In [ ]:
temp['temperature'] = quantile(temp,'temperature')


In [ ]:
temp['humidity'] = quantile(temp,'humidity')


In [ ]:
temp['ph'] = quantile(temp,'ph')


In [ ]:
temp['rainfall'] = quantile(temp,'rainfall')


In [ ]:
temp.head()

In [ ]:
temp = temp.dropna()


In [ ]:
temp.isnull().sum()

In [ ]:
temp.shape

In [ ]:
temp.head()

In [ ]:
for i in temp.columns[:-1]:
    print(f' Statistical plots for {i} is shown ↓')
    visualise(temp,i)
    print("=>"*50)

In [ ]:
data = df.copy()

In [ ]:
data['N'],params = stats.boxcox(data['N']+1)

In [ ]:
stats.probplot(data['N'],dist='norm',plot=plt)

In [ ]:
data = temp.copy()

In [ ]:
data['N'] = data.N**(1/1.2)

In [ ]:
stats.probplot(data['N'],dist='norm',plot=plt)

In [ ]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [ ]:
temp.head(2)

In [ ]:
test = temp['label']

In [ ]:
train = temp.drop(['label'],axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train,test,test_size = 0.2)

In [ ]:
sc = StandardScaler()


In [ ]:
train_scaled = sc.fit_transform(X_train)

In [ ]:
test_scaled = sc.transform(X_test)

In [ ]:
scaled_df = pd.DataFrame(train_scaled,columns=train.columns)

In [ ]:

for i in scaled_df.columns:
    print(f' Statistical plots for {i} is shown ↓')
    visualise(scaled_df,i)
    print("=>"*50)

In [ ]:
scaled_df.head()

In [ ]:
mm = MinMaxScaler()


In [ ]:
train_mm = mm.fit_transform(X_train)


In [ ]:
test_mm = mm.transform(X_test)

In [ ]:
scaled_mm = pd.DataFrame(train_mm,columns=train.columns)

In [ ]:
for i in scaled_mm.columns:
    print(f' Statistical plots for {i} is shown ↓')
    visualise(scaled_mm,i)
    print("=>"*50)

In [ ]:
scaled_mm.head()

In [ ]:
dum = scaled_df.copy()

In [ ]:
dum['N'] = np.log(dum['N']+1)

In [ ]:
stats.probplot(dum['N'],dist = 'norm',plot=plt)

In [ ]:
X_train_scaled = mm.fit_transform(X_train)

In [ ]:
X_test_scaled = mm.fit_transform(X_test)

### ML ALGO 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV,cross_val_score

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [ ]:
from sklearn import metrics


In [ ]:
from sklearn.svm import SVC 

In [ ]:
from sklearn import tree

In [ ]:
features = df[['N', 'P','K','temperature', 'humidity', 'ph', 'rainfall']]

In [ ]:
target = df['label']

In [ ]:
#features = df[['temperature', 'humidity', 'ph', 'rainfall']]
labels = df['label']

In [ ]:
# Initialzing empty lists to append all model's name and corresponding name
acc = []
model = []

In [ ]:
# Splitting into train and test data

from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features,target,test_size = 0.2,random_state =2)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DecisionTree = DecisionTreeClassifier(criterion="entropy",random_state=2,max_depth=5)

In [ ]:
DecisionTree.fit(Xtrain,Ytrain)

In [ ]:
predicted_values = DecisionTree.predict(Xtest)

In [ ]:
x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:
acc.append(x)

In [ ]:
model.append('DT')

In [ ]:
print("DecisionTrees's Accuracy is: ", x*100)

In [ ]:
print(classification_report(Ytest,predicted_values))

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
features

In [ ]:
# Cross validation score (Decision Tree)
score = cross_val_score(DecisionTree, features, target,cv=5)

In [ ]:
score

## Extra Tree Classifier

In [ ]:
ExtraTreeClassifier = tree.ExtraTreeClassifier()

In [ ]:
ExtraTreeClassifier.fit(Xtrain,Ytrain)

In [ ]:
predicted_values = ExtraTreeClassifier.predict(Xtest)

In [ ]:
x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:
acc.append(x)
model.append('ETC')

In [ ]:
print("ExtraTreeClassifier's Accuracy is: ", x*100)

In [ ]:

print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (Decision Tree)
score = cross_val_score(ExtraTreeClassifier, features, target,cv=5)

In [ ]:
score

### Saving trained Decision Tree model

In [ ]:
import pickle

In [ ]:
# Dump the trained Naive Bayes classifier with Pickle
DT_pkl_filename = '../models/DecisionTree.pkl'

In [ ]:
# Open the file to save as pkl file
DT_Model_pkl = open(DT_pkl_filename, 'wb')


In [ ]:
pickle.dump(DecisionTree, DT_Model_pkl)

In [ ]:
# Close the pickle instances
DT_Model_pkl.close()

# Guassian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
NaiveBayes = GaussianNB()

In [ ]:
NaiveBayes.fit(Xtrain,Ytrain)

In [ ]:
predicted_values = NaiveBayes.predict(Xtest)

In [ ]:
x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:
acc.append(x)

In [ ]:
model.append('NB')

In [ ]:
print("Naive Bayes's Accuracy is: ", x)

In [ ]:
print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (NaiveBayes)
score = cross_val_score(NaiveBayes,features,target,cv=5)

In [ ]:
score

### Saving trained Guassian Naive Bayes model

In [ ]:
# Dump the trained Naive Bayes classifier with Pickle
NB_pkl_filename = '../models/NBClassifier.pkl'


In [ ]:
# Open the file to save as pkl file
NB_Model_pkl = open(NB_pkl_filename, 'wb')
pickle.dump(NaiveBayes, NB_Model_pkl)

In [ ]:
# Close the pickle instances
NB_Model_pkl.close()

# Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC

In [ ]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# fit scaler on training data
norm = MinMaxScaler().fit(Xtrain)

In [ ]:
X_train_norm = norm.transform(Xtrain)

In [ ]:
# transform testing dataabs
X_test_norm = norm.transform(Xtest)

In [ ]:
SVM = SVC(kernel='poly', degree=3, C=1)
SVM.fit(X_train_norm,Ytrain)

In [ ]:
predicted_values = SVM.predict(X_test_norm)

In [ ]:
x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:
acc.append(x)
model.append('SVM')

In [ ]:
print("SVM's Accuracy is: ", x)

In [ ]:
print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (SVM)
score = cross_val_score(SVM,features,target,cv=5)


In [ ]:
score

In [ ]:
#Saving trained SVM model

In [ ]:
# Dump the trained SVM classifier with Pickle
SVM_pkl_filename = '../models/SVMClassifier.pkl'

In [ ]:
# Open the file to save as pkl file
SVM_Model_pkl = open(SVM_pkl_filename, 'wb')
pickle.dump(SVM, SVM_Model_pkl)

In [ ]:
# Close the pickle instances
SVM_Model_pkl.close()

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LogReg = LogisticRegression(random_state=2)

In [ ]:
LogReg.fit(Xtrain,Ytrain)

In [ ]:
predicted_values = LogReg.predict(Xtest)

In [ ]:
x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:
acc.append(x)
model.append('Logistic Regression')

In [ ]:
print("Logistic Regression's Accuracy is: ", x)


In [ ]:

print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (Logistic Regression)
score = cross_val_score(LogReg,features,target,cv=5)

In [ ]:
score

### Saving trained Logistic Regression model

In [ ]:

# Dump the trained Naive Bayes classifier with Pickle
LR_pkl_filename = '../models/LogisticRegression.pkl'

In [ ]:
# Open the file to save as pkl file
LR_Model_pkl = open(DT_pkl_filename, 'wb')
pickle.dump(LogReg, LR_Model_pkl)


In [ ]:
# Close the pickle instances
LR_Model_pkl.close()

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=20, random_state=0)

In [ ]:
RF.fit(Xtrain,Ytrain)


In [ ]:

predicted_values = RF.predict(Xtest)


In [ ]:

x = metrics.accuracy_score(Ytest, predicted_values)


In [ ]:
acc.append(x)
model.append('RF')


In [ ]:
print("RF's Accuracy is: ", x)


In [ ]:

print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (Random Forest)
score = cross_val_score(RF,features,target,cv=5)


In [ ]:
score

### Saving trained Random Forest model

In [ ]:
# Dump the trained Naive Bayes classifier with Pickle
RF_pkl_filename = '../models/RandomForest.pkl'

In [ ]:
# Open the file to save as pkl file
RF_Model_pkl = open(RF_pkl_filename, 'wb')
pickle.dump(RF, RF_Model_pkl)


In [ ]:
# Close the pickle instances
RF_Model_pkl.close()

# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
XB = xgb.XGBClassifier()

In [ ]:
XB.fit(Xtrain,Ytrain)

In [ ]:

predicted_values = XB.predict(Xtest)

In [ ]:

x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:

acc.append(x)
model.append('XGBoost')

In [ ]:

print("XGBoost's Accuracy is: ", x)

In [ ]:
print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (XGBoost)
score = cross_val_score(XB,features,target,cv=5)


In [ ]:
score

### Saving trained XGBoost model

In [ ]:
# Dump the trained Naive Bayes classifier with Pickle
XB_pkl_filename = '../models/XGBoost.pkl'


In [ ]:
# Open the file to save as pkl file
XB_Model_pkl = open(XB_pkl_filename, 'wb')
pickle.dump(XB, XB_Model_pkl)

In [ ]:
# Close the pickle instances
XB_Model_pkl.close()

## GradientBoosting Classifier

In [ ]:
GBC = GradientBoostingClassifier(n_estimators=20, random_state=0)

In [ ]:
GBC.fit(Xtrain,Ytrain)

In [ ]:

predicted_values = GBC.predict(Xtest)

In [ ]:
x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:
acc.append(x)
model.append('GBC')

In [ ]:
print("GBC's Accuracy is: ", x)

In [ ]:
print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (GradientBoostingClassifier)
score = cross_val_score(GBC,features,target,cv=5)


In [ ]:
score

## K Nearest Neighbors

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# fit scaler on training data
norm = MinMaxScaler().fit(Xtrain)
X_train_norm = norm.transform(Xtrain)

In [ ]:
# transform testing dataabs
X_test_norm = norm.transform(Xtest)

In [ ]:
KNN = KNeighborsClassifier()

In [ ]:
KNN.fit(X_train_norm,Ytrain)

In [ ]:
predicted_values = KNN.predict(X_test_norm)

In [ ]:
x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:
acc.append(x)
model.append('KNN')

In [ ]:
print("KNN's Accuracy is: ", x)

In [ ]:

print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (GradientBoostingClassifier)
score = cross_val_score(KNN,features,target,cv=5)


In [ ]:
score

## Stochastic Gradient Descent Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
SGD=SGDClassifier(loss='modified_huber',shuffle=True,random_state=101)

In [ ]:
SGD.fit(X_train_norm,Ytrain)

In [ ]:
predicted_values = SGD.predict(X_test_norm)

In [ ]:
x = metrics.accuracy_score(Ytest, predicted_values)

In [ ]:
acc.append(x)
model.append('SGD')

In [ ]:

print("SGD's Accuracy is: ", x)


In [ ]:

print(classification_report(Ytest,predicted_values))

In [ ]:
# Cross validation score (Logistic Regression)
score = cross_val_score(SGD,features,target,cv=5)
score

In [ ]:
accuracy_models = dict(zip(model, acc))

In [ ]:
plt.figure(figsize=[10,5],dpi = 100)
plt.title('Accuracy Comparison')
plt.xlabel('Accuracy')
plt.ylabel('Algorithm')
sns.barplot(x =model,y = acc,palette='dark',orient='v')

In [ ]:
for k, v in accuracy_models.items():
    print (k, '-->', v)

## Making a prediction

In [ ]:
data = np.array([[104,18, 30, 23.603016, 60.3, 6.7, 140.91]])

In [ ]:
prediction = RF.predict(data)

In [ ]:
print(prediction)

In [ ]:
data = np.array([[83, 45, 60, 28, 70.3, 7.0, 150.9]])

In [ ]:
prediction = RF.predict(data)

In [ ]:
print(prediction)

## Hyper Parameter tuning

In [ ]:
params = {
    'RandomForest':{
        'model': RandomForestClassifier(),
        'params':{
            'n_estimators': [int(x) for x in np.linspace(start=1,stop=1200,num=10)],
            'max_depth':[int(x) for x in np.linspace(start=1,stop=30,num=5)],
            'min_samples_split':[2,5,10,12],
            'min_samples_leaf':[2,5,10,12],
            'max_features':['auto','sqrt'],
            'ccp_alpha':[0.030,0.035,0.040,0.045,0.050],
        }
    },
    
    'logistic':{
        'model':LogisticRegression(),
        'params':{
            'penalty':['l1', 'l2', 'elasticnet'],
            'C':[0.25,0.50,0.75,1.0],
            'tol':[1e-10,1e-5,1e-4,1e-3,0.025,0.25,0.50],
            'solver':['lbfgs','liblinear','saga','newton-cg'],
            'multi_class':['auto', 'ovr', 'multinomial'],
            'max_iter':[int(x) for x in np.linspace(start=1,stop=250,num=10)],
        }
    },
    'D-tree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion':['gini','entropy'],
            'splitter':['best','random'],
            'min_samples_split':[1,2,5,10,12],
            'min_samples_leaf':[1,2,5,10,12],
            'max_features':['auto','sqrt'],
            'ccp_alpha':[0.030,0.035,0.040,0.045,0.050],
        }
    },
    'SVM':{
        'model':SVC(),
        'params':{
            'C':[0.25,0.50,0.75,1.0],
            'tol':[1e-10,1e-5,1e-4,0.025,0.50,0.75],
            'kernel':['linear','poly','sigmoid','rbf'],
            'max_iter':[int(x) for x in np.linspace(start=1,stop=250,num=10)],
        }
    }
}


In [ ]:

scores = []
for model_name,mp in params.items():
    clf = RandomizedSearchCV(mp['model'],param_distributions= mp['params'],cv = 5,n_jobs=-1,n_iter = 10,scoring='accuracy',
                            verbose=2)
    clf.fit(X_train,y_train)
    scores.append({
        'model_name':model_name,
        'best_score':clf.best_score_,
        'best_estimator':clf.best_estimator_
    })


In [ ]:
scores_df = pd.DataFrame(data = scores,columns = ['model_name','best_estimator','best_score'])
scores_df


In [ ]:
for i in scores_df['best_estimator']:
    print(i)

In [ ]:
rf = RandomForestClassifier(ccp_alpha=0.03, max_depth=22, min_samples_leaf=12,
                       min_samples_split=5, n_estimators=800)

In [ ]:
lr = LogisticRegression(C=0.75, max_iter=194, multi_class='ovr', penalty='l1',
                   solver='liblinear')


In [ ]:

svc = SVC(C=0.5, kernel='poly', max_iter=139, tol=1e-10)


In [ ]:

rf_val = cross_val_score(estimator=rf,X=X_train_norm,y=Ytrain,cv=20,n_jobs=-1)


In [ ]:
lr_val = cross_val_score(estimator = lr,X=X_train_norm,y=Ytrain,cv=20,n_jobs=-1)


In [ ]:
svc_val = cross_val_score(estimator=svc,X=X_train_norm,y=Ytrain,cv=20,n_jobs=-1)  

In [ ]:
print(f"Random Forest Validation-Score: {np.round(np.mean(rf_val)*100,1)}",)

In [ ]:
print(f"Logistic Regression Validation-Score: {np.round(np.mean(lr_val)*100,1)}%",)

In [ ]:
print(f"Support Vector Classifier Validation-Score: {np.round(np.mean(svc_val)*100,1)}%",)

In [ ]:
svc = SVC(C=0.5, kernel='poly', max_iter=139, tol=1e-10)
svc.fit(X_train_norm,Ytrain)
svc.score(X_train_norm,Ytrain)

In [ ]:
rf = RandomForestClassifier(ccp_alpha=0.03, max_depth=22, min_samples_leaf=12,
                       min_samples_split=5, n_estimators=800)

rf.fit(X_train_norm,Ytrain)
rf.score(X_train_norm,Ytrain)

In [ ]:
lr = LogisticRegression(C=0.75, max_iter=194, multi_class='ovr', penalty='l1',
                   solver='liblinear')
lr.fit(X_train_norm,Ytrain)
lr.score(X_train_norm,Ytrain)